# <b>Start here</b> - Initial environment setup
Ensure to run each step in order - Optional steps may be skipped if desired

In [1]:
#@title Check which Nvidia GPU has been assigned to you
!nvidia-smi

Thu May 19 11:48:22 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#@title Step 1 - Set up the environment (Mandatory)
onGoogleColab = True
onWindows = False
previouslySetup = False
GPUID = 0

import os

ffmpeg4 = 'ffmpeg'
nvencPresetLocal = 'p7'
if onGoogleColab:
    !wget https://johnvansickle.com/ffmpeg/releases/ffmpeg-release-amd64-static.tar.xz
    !tar -xf ffmpeg-release-amd64-static.tar.xz
    !rm ffmpeg-release-amd64-static.tar.xz

    ffmpegdir = 'ffmpeg-4.3.1-amd64-static'
    dirContent = os.listdir()
    for objName in dirContent:
        if 'ffmpeg' in objName:
            ffmpegdir = objName
            print('Found ffmpeg dir:',ffmpegdir)
    ffmpeg4 = '/content/'+ffmpegdir+'/ffmpeg'
    nvencPresetLocal = 'slow'

print('NVENC PRESET',nvencPresetLocal)
!git clone --branch main --recurse-submodules https://github.com/HeylonNHP/RIFE-Colab.git RIFEcolab

os.chdir('RIFEcolab/')

import sys

# Why should we need to add the submodule to the path, just for the RIFE import to work
# Thanks for being consistently terrible, python
sys.path.insert(0, os.getcwd() + os.path.sep + 'arXiv2020RIFE')

from generalInterpolationProceedures import *
setGPUinterpolationOptions(3,[0])
setNvencSettings(GPUID,nvencPresetLocal)
setFFmpeg4Path(ffmpeg4)
print('NVENC PRESET',nvencPresetLocal)

--2022-05-19 11:48:34--  https://johnvansickle.com/ffmpeg/releases/ffmpeg-release-amd64-static.tar.xz
Resolving johnvansickle.com (johnvansickle.com)... 107.180.57.212
Connecting to johnvansickle.com (johnvansickle.com)|107.180.57.212|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40846104 (39M) [application/x-xz]
Saving to: ‘ffmpeg-release-amd64-static.tar.xz’

ffmpeg-release-amd6 100%[===================>]  38.95M  17.4MB/s    in 2.2s    

2022-05-19 11:48:36 (17.4 MB/s) - ‘ffmpeg-release-amd64-static.tar.xz’ saved [40846104/40846104]

Found ffmpeg dir: ffmpeg-5.0.1-amd64-static
NVENC PRESET slow
Cloning into 'RIFEcolab'...
remote: Enumerating objects: 1048, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 1048 (delta 2), reused 5 (delta 1), pack-reused 1038
Receiving objects: 100% (1048/1048), 267.46 KiB | 6.86 MiB/s, done.
Resolving deltas: 100% (675/675), done.
Submodule 'arXiv2020RIFE' (htt

In [3]:
#@title Step 2 - Mount your google drive to use as storage for input/output videos (Optional)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Processing a single video (Extract -> Interpolate -> Create output - All at once)
**Processing a single video has now been broken into two steps:**<br>
Run the **set settings for interpolation** step first to apply the desired settings for interpolation, then run the second step to start the interpolator. <br><br>
Paste in the path to your video, choose your desired settings, then run this code box to interpolate a single video
<br>
The output will be in the same folder as the input
<br><br>
<<< - Click the folder icon on the left hand side to open the file browser<br>
Navigate to the file you'd like to interpolate<br>
Right click it, click Copy Path and paste it over the top of the example path below

In [4]:
#@title Run to set settings for interpolation (DO FIRST)
#@markdown Input video path
inputFile = '/content/drive/MyDrive/dataset/Animation/NewEdition/TurtleProjecttest/Video/Video.mp4' #@param{type:"string"}
#@markdown Frame handling mode (Available choices: 1, 3, or 4. Equivalent to DAINAPP)
mode =  3#@param{type:"number"}
#@markdown Enable perfect loop (For GIFs or looping videos)
loopable = True #@param{type:"boolean"}
#@markdown Interpolation factor (2, 4, 8, 16, 32... etc)
interpolationFactor = 16 #@param{type:"number"}
#@markdown x264 CRF value for producing the output
crf = 20 #@param{type:"number"}
#@markdown Clear the original_frames and interpolated frames after completion
clearPNGs = True #@param{type:"boolean"}
#@markdown Don't store the PNG files in the same folder as the input video
nonLocalPNGs = True #@param{type:"boolean"}
#@markdown Scenechange sensitivity (lower is more sensitive)
scenechangeSensitivity = 0.20 #@param{type:"number"}
#@markdown Sensitivity of mpdecimate when detecting and removing duplicate frames [hi,lo,frac] (mode 3/4)
mpdecimateSensitivity = "64*12,64*8,0.33" #@param{type:"string"}
#@markdown Use NVENC for output
useNvenc = True #@param{type:"boolean"}
#@markdown Use auto encode
autoEncode = True #@param{type:"boolean"}
#@markdown Auto encode block size
autoBlockSize = 3000 #@param{type:"number"}
#@markdown Enable mode 3 target FPS
mode3TargetFPSEnable = True #@param{type:"boolean"}
#@markdown Mode 3 target FPS:
mode3TargetFPSValue =  24#@param{type:"number"}

encoderConfig = EncoderConfig()
encoderConfig.setEncodingCRF(float(crf))
if bool(useNvenc):
    encoderConfig.enableNvenc(True)
    encoderConfig.setEncodingPreset('slow')
encoderConfig.setNvencGPUID(GPUID)

interpolatorConfig = InterpolatorConfig()

interpolatorConfig.setMode(mode)
interpolatorConfig.setClearPngs(clearPNGs)
interpolatorConfig.setNonlocalPngs(nonLocalPNGs)
interpolatorConfig.setLoopable(loopable)
interpolatorConfig.setInterpolationFactor(interpolationFactor)
interpolatorConfig.setMpdecimateSensitivity(mpdecimateSensitivity)
interpolatorConfig.setScenechangeSensitivity(scenechangeSensitivity)
interpolatorConfig.setUhdScale(1.0)
interpolatorConfig.setAccountForDuplicateFrames(True)
interpolatorConfig.setMode3TargetFPS(mode3TargetFPSEnable,mode3TargetFPSValue)

In [5]:
#@title Interpolate a single video - Run interpolator (DO SECOND)
performAllSteps(inputFile, interpolatorConfig, encoderConfig, useAutoEncode=autoEncode,autoEncodeBlockSize=autoBlockSize)

ffmpeg version 5.0.1-static https://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 8 (Debian 8.3.0-6)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-debug --disable-ffplay --disable-indev=sndio --disable-outdev=sndio --cc=gcc --enable-fontconfig --enable-frei0r --enable-gnutls --enable-gmp --enable-libgme --enable-gray --enable-libaom --enable-libfribidi --enable-libass --enable-libvmaf --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librubberband --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libvorbis --enable-libopus --enable-libtheora --enable-libvidstab --enable-libvo-amrwbenc --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libdav1d --enable-libxvid --enable-libzvbi --enable-libzimg
  libavutil      57. 17.100 / 57. 17.100
  libavcodec     59. 18.100 / 59. 18.100
  libavformat   

  0%|          | 0/106 [00:00<?, ?it/s]

LOADED ALL FRAMES - DONE


 96%|█████████▌| 102/106 [00:35<00:01,  3.77it/s]

END


 99%|█████████▉| 105/106 [00:35<00:00,  4.37it/s]

END
END
Put none - ask save thread to quit
Got none - save thread
Got none - save thread


 99%|█████████▉| 105/106 [00:36<00:00,  2.91it/s]


Got none - save thread
---INTERPOLATION DONE---
Auto encode block 1 2240 3497916ms Before 000000000000000.png Start 000000000002083.png End 000000003497916.png
KEEP THIS FRAME 000000003497916.png
------LINES------
  Duration: 00:00:35.00, start: 0.000000, bitrate: 295 kb/s
---AUTO ENCODING DONE---


# Processing a single video (Run steps individually)
Please run **Run to set settings for interpolation (DO FIRST)** first, before running the steps below in order **Extract frames -> Run interpolator -> Create output**
<br><br>
Running the steps individually like this will allow you to extract the frames, check for duplicate frames that were missed by FFmpeg after extraction and manually delete them, then run the interpolator, and check for any missed scene changes in the interpolated frames before creating the output

In [ ]:
#@title Extract frames (step 1)
performAllSteps(inputFile, interpolatorConfig, encoderConfig, useAutoEncode=autoEncode,autoEncodeBlockSize=autoBlockSize,step1=True,step2=False,step3=False)

In [ ]:
#@title Run interpolator (step 2)
performAllSteps(inputFile, interpolatorConfig, encoderConfig, useAutoEncode=autoEncode,autoEncodeBlockSize=autoBlockSize,step1=False,step2=True,step3=False)

In [ ]:
#@title Create output (step 3)
performAllSteps(inputFile, interpolatorConfig, encoderConfig, useAutoEncode=autoEncode,autoEncodeBlockSize=autoBlockSize,step1=False,step2=False,step3=True)

# Processing multiple videos using the batch interpolator
set inputDirectory to equal the folder path containing the input videos
<br><br>
Each video must be in its own subdirectory within the input folder
<br><br>
The interpolation factor for each video is chosen based on its input FPS (Accounting for duplicate frames if using mode 4). It will choose an interpolation factor which will meet or exceed the 'Output FPS target'
<br><br>
For perfectly looping GIFs/videos: Add [l] to the filename or subfolder name of the input file. E.G.:<br>
inputLoopingGifFileNumberOne[l].gif
<br>
or
<br>
InputLoopingGifFolder[l]/inputLoopingGifFileNumberOne.gif

In [ ]:
#@title Interpolate a folder of videos
#@markdown Input folder path
inputDirectory = 'input.mp4' #@param{type:"string"}
#@markdown Frame handling mode (Available choices: 1, 3, or 4. Equivalent to DAINAPP)
mode = 3 #@param{type:"number"}
#@markdown Output FPS target
fpsTarget = 60 #@param{type:"number"}
#@markdown x264 CRF value for producing the output
crf = 20 #@param{type:"number"}
#@markdown Clear the original_frames and interpolated frames after completion
clearPNGs = True #@param{type:"boolean"}
#@markdown Don't store the PNG files in the same folder as the input video
nonLocalPNGs = True #@param{type:"boolean"}
#@markdown Scenechange sensitivity (lower is more sensitive)
scenechangeSensitivity = 0.20 #@param{type:"number"}
#@markdown Sensitivity of mpdecimate when detecting and removing duplicate frames [hi,lo,frac] (mode 3/4)
mpdecimateSensitivity = "64*12,64*8,0.33" #@param{type:"string"}
#@markdown Use NVENC for output
useNvenc = True #@param{type:"boolean"}
#@markdown Use auto encode
autoEncode = False #@param{type:"boolean"}
#@markdown Auto encode block size
autoBlockSize = 3000 #@param{type:"number"}
#@markdown Enable mode 3 target FPS
mode3TargetFPSEnable = True #@param{type:"boolean"}
#@markdown Mode 3 target FPS:
mode3TargetFPSValue = 60.000 #@param{type:"number"}


encoderConfig = EncoderConfig()
encoderConfig.setEncodingCRF(float(crf))
if bool(useNvenc):
    encoderConfig.enableNvenc(True)
    encoderConfig.setEncodingPreset('slow')
encoderConfig.setNvencGPUID(GPUID)
encoderConfig.setLoopingOptions(10,15,True)

interpolatorConfig = InterpolatorConfig()

interpolatorConfig.setMode(mode)
interpolatorConfig.setClearPngs(clearPNGs)
interpolatorConfig.setNonlocalPngs(nonLocalPNGs)
# interpolatorConfig.setLoopable(args.loopable)
# interpolatorConfig.setInterpolationFactor(args.interpolationFactor)
interpolatorConfig.setMpdecimateSensitivity(mpdecimateSensitivity)
interpolatorConfig.setScenechangeSensitivity(scenechangeSensitivity)
interpolatorConfig.setUhdScale(1.0)
interpolatorConfig.setAccountForDuplicateFrames(True)
interpolatorConfig.setMode3TargetFPS(mode3TargetFPSEnable,mode3TargetFPSValue)

batchInterpolateFolder(inputDirectory,interpolatorConfig,fpsTarget,encoderConfig,useAutoEncode=autoEncode,autoEncodeBlockSize=autoBlockSize)